In [1]:
import torch
import random
import numpy as np
import os

from tqdm import tqdm, trange
# torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_pretrained_bert.optimization import BertAdam
# from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer

In [2]:
from run_classifier import ColaProcessor, MrpcProcessor, logger, convert_examples_to_features,\
    set_optimizer_params_grad, copy_optimizer_params_to_model, accuracy, p_r_f1, tp_pcount_gcount, convert_claims_to_features, convert_pers_to_features

In [3]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()
#     device = torch.device("cpu")
    print('There are %d GPU(s) available.' % n_gpu)

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1050 Ti


In [4]:
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertPreTrainedModel, BertModel, BertConfig
from torch.nn import BCEWithLogitsLoss, CosineEmbeddingLoss,CrossEntropyLoss, MSELoss

05/27/2020 23:20:24 - INFO - transformers.file_utils -   PyTorch version 1.4.0 available.
05/27/2020 23:22:13 - INFO - transformers.file_utils -   TensorFlow version 2.1.0 available.


In [5]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [6]:
# def train_and_test(data_dir, bert_model="bert-base-uncased", task_name=None,
#                    output_dir=None, max_seq_length=128, do_train=False, do_eval=False, do_lower_case=False,
#                    train_batch_size=32, eval_batch_size=8, learning_rate=5e-5, num_train_epochs=3,
#                    warmup_proportion=0.1,no_cuda=False, local_rank=-1, seed=42, gradient_accumulation_steps=1,
#                    optimize_on_cpu=False, fp16=False, loss_scale=128, saved_model=""):

In [5]:
class BertForConsistencyCueClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels=2):
        super(BertForConsistencyCueClassification, self).__init__(config)
        self.num_labels = num_labels

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size*3, num_labels)
        self.classifier2 = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)
#         self.init_weights()

#     @add_start_docstrings_to_callable(BERT_INPUTS_DOCSTRING)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        input_ids2=None,
        attention_mask2=None,
        token_type_ids2=None,
        position_ids2=None,
        head_mask2=None,
        inputs_embeds2=None,
        labels2=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import BertTokenizer, BertForSequenceClassification
        import torch

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)

        loss, logits = outputs[:2]

        """

        _, outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             head_mask=head_mask,
#             inputs_embeds=inputs_embeds,
        )

        _, outputsC = self.bert(
            input_ids2,
            attention_mask=attention_mask2,
            token_type_ids=token_type_ids2,
#             position_ids=position_ids2,
#             head_mask=head_mask2,
#             inputs_embeds=inputs_embeds2,
        )
#         print("Careful, outputs:")
#         print(outputs)
#         print(outputsC)
        pooled_output = outputs
        pooled_outputC = outputsC

        pooled_output = self.dropout(pooled_output)
        pooled_outputC = self.dropout(pooled_outputC)
        
#         cos_pooled_outputs = torch.cosine_similarity(pooled_output, pooled_outputC, dim=1)
        final_output_cat = torch.cat((pooled_output, pooled_outputC))
        final_output_minus = torch.abs(pooled_output-pooled_outputC)
#         final_output_mult = torch.mul(pooled_output, pooled_outputC)
#         final_output_mimu = torch.cat((final_output_minus, final_output_mult))
#         final_output_camu = torch.cat((final_output_cat, final_output_mult))
        final_output_cami = torch.cat((final_output_cat, final_output_minus))
#         final_output_camimu = torch.cat((final_output_cat, final_output_minus, final_output_mult))
#         1
#         torch.Size([64, 768])
#         2
#         torch.Size([32, 768])
#         3
#         torch.Size([32, 768])
#         4
#         torch.Size([64, 768])
#         5
#         torch.Size([96, 768])
#         6
#         torch.Size([96, 768])
#         7
#         torch.Size([128, 768])
#         print(1)
#         print(final_output_cat.size())
#         print(2)
#         print(final_output_minus.size())
#         print(3)
#         print(final_output_mult.size())
#         print(4)
#         print(final_output_mimu.size())
#         print(5)
#         print(final_output_camu.size())
#         print(6)
#         print(final_output_cami.size())
#         print(7)
#         print(final_output_camimu.size())
        
        
#         print('pooled_output size:')
#         print(pooled_output.size())
#         print(pooled_output)
#         print('cos_pooled_outputs size:')
#         print(cos_pooled_outputs.size())
#         print(cos_pooled_outputs)
        batch_size = list(pooled_output.size())[0]
        hidden_size = list(pooled_output.size())[1]

        logits_ce = self.classifier2(pooled_output)
        print('logits_ce:')
        print(logits_ce)
        
    
        ## v2: concat
        ## v3: multiply
        ## v4: v2 & ce_cos_similarity
        ## v5: v3 & ce_cos_similarity
        
#         print(torch.cat((pooled_output, cos_pooled_outputs.unsqueeze(1)),1))
#         print((pooled_output*cos_pooled_outputs.unsqueeze(1)))
#         logits_cos = self.classifier(torch.cat((pooled_output, cos_pooled_outputs.unsqueeze(1)),1))
#         logits_cos = self.classifier2((pooled_output*cos_pooled_outputs.unsqueeze(1)))
#         self.classifier = torch.nn.Linear(hidden_size+batch_size, 2).to(device)
#         logits_cos = self.classifier(torch.cat((pooled_output, cos_pooled_outputs.repeat(batch_size,1)),1))
#         print('logits_cos:')
#         print(logits_cos)
        
#         logits = self.classifier(pooled_output)
#         logitsC = self.classifier(pooled_outputC)

#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
#         print(logits)
#         print('xd')
#         print(outputs[2:])
#         outputs = (logits,) + outputs[2:]
#         print("labels:")
#         print(labels)
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
#                 loss_fct_ce = CrossEntropyLoss()
#                 loss_ce = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

                loss_fct_ce = CrossEntropyLoss()
#                 print('pooled_output size:')
#                 print(pooled_output.size())
                loss_ce = loss_fct_ce(logits_ce.view(-1, self.num_labels), labels.view(-1))
#                 loss_ce = loss_fct_ce(pooled_output.view(-1), labels.view(-1))
                print('loss_ce:')
                print(loss_ce)

                
#                 loss_fct_cos = CosineEmbeddingLoss()
# #                 print(labels)
# #                 print(pooled_outputC)


#                 labels2[labels2==0] = -1
#                 loss_cos = loss_fct_cos(pooled_output, pooled_outputC, labels2)
#                 labels2[labels2==-1] = 0
        
# #                 loss_cos = loss_fct_cos(logits_ce, logits_cos, labels2)
# #                 loss_cos = loss_fct_ce(logits_cos.view(-1, self.num_labels), labels2.view(-1))
#                 print('loss_cos:')
#                 print(loss_cos)
            
                loss = loss_ce
                print('final loss:')
                print(loss)
#                 logits = self.classifier(loss)
#             outputs = (loss,) + outputs
#             outputs = (loss,) + logits_cos 
                outputs = loss
                return outputs
        else:
            return logits_ce
        
          # (loss), logits, (hidden_states), (attentions)


In [8]:
# labe = torch.ones(8)
# print(labe )
# labe[labe==1] = -1
# labe

In [9]:
# a = torch.tensor([[4.2,5,6,7,8],[9,10,11,12,13]])
# b = torch.tensor([[3.4,5,6,7,8],[9,10,11,12,13]])

# # a = torch.tensor([[4,5,6,7,8],[9,10,11,12,13]])
# # b = torch.tensor([1,2])
# c= torch.cosine_similarity(a,b,dim=1)
# # b.unsqueeze(1)
# # # a.size()
# # torch.cat((a,c.unsqueeze(1)),1)
# a*c.unsqueeze(1)

In [10]:
# input1 = torch.randn(100, 128)
# input2 = torch.randn(100, 128)
# output = torch.cosine_similarity(input1, input2, dim=1)

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

05/27/2020 22:52:48 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\arsen\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [12]:
# Prepare model 
model = BertForConsistencyCueClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# model = BertModel.from_pretrained('bert-base-uncased')

05/27/2020 22:52:49 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\arsen\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/27/2020 22:52:49 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file C:\Users\arsen\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\arsen\AppData\Local\Temp\tmp30h_iksc
05/27/2020 22:52:53 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_lay

BertForConsistencyCueClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fals

In [13]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 203 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [12]:
data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
data_dir_output = "D:/Projects/Stance/Models/Consistency_Cues/"
output_dir=data_dir_output
max_seq_length=32
max_grad_norm = 1.0
num_training_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1
# warmup_proportion = 0.1
# train_batch_size=32
train_batch_size=16
eval_batch_size=8
learning_rate=5e-5
num_train_epochs=3
local_rank=-1
seed=19
gradient_accumulation_steps=1
loss_scale=128
train_batch_size = int(train_batch_size / gradient_accumulation_steps)

processors = {
        "mrpc": MrpcProcessor,
    }

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
    
os.makedirs(output_dir, exist_ok=True)
processor = processors['mrpc']()
label_list = processor.get_labels()
num_labels = len(label_list)
# print('label list')
# print(label_list)

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(
    len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

##preprare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]
t_total = num_train_steps
optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)
# optimizer = AdamW(optimizer_grouped_parameters,
#                   lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
#                   correct_bias=False
#                 )

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)  # PyTorch scheduler

05/27/2020 23:26:58 - INFO - run_classifier -   LOOKING AT D:/Jupyter/data/dataset/perspective_stances/train.tsv


NameError: name 'model' is not defined

In [15]:
global_step = 0
claim_features = convert_claims_to_features(train_examples, label_list, max_seq_length, tokenizer)
train_features = convert_pers_to_features(train_examples, label_list, max_seq_length, tokenizer)

logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)


all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

claims_input_ids = torch.tensor([f.input_ids for f in claim_features], dtype=torch.long)
claims_input_mask = torch.tensor([f.input_mask for f in claim_features], dtype=torch.long)
claims_segment_ids = torch.tensor([f.segment_ids for f in claim_features], dtype=torch.long)
claims_label_ids = torch.tensor([f.label_id for f in claim_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, claims_input_ids, claims_input_mask, claims_segment_ids, claims_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

# claims_data = TensorDataset(claims_input_ids, claims_input_mask, claims_segment_ids, claims_label_ids)
# claims_sampler = RandomSampler(claims_data)
# claims_dataloader = DataLoader(claims_data, sampler=train_sampler, batch_size=train_batch_size)

05/27/2020 22:52:57 - INFO - run_classifier -   *** Claim Example ***
05/27/2020 22:52:57 - INFO - run_classifier -   guid: train-1
05/27/2020 22:52:57 - INFO - run_classifier -   tokens: [CLS] male infant ci ##rc ##um ##cision is tan ##tam ##ount to child abuse [SEP]
05/27/2020 22:52:57 - INFO - run_classifier -   input_ids: 101 3287 10527 25022 11890 2819 28472 2003 9092 15464 21723 2000 2775 6905 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/27/2020 22:52:57 - INFO - run_classifier -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/27/2020 22:52:57 - INFO - run_classifier -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/27/2020 22:52:57 - INFO - run_classifier -   label: 0 (id = 0)
05/27/2020 22:52:57 - INFO - run_classifier -   *** Claim Example ***
05/27/2020 22:52:57 - INFO - run_classifier -   guid: train-2
05/27/2020 22:52:57 - INFO - run_classifier -   tokens: [CLS] male infant ci ##rc ##um ##cision is tan ##tam ##o

In [16]:
model.train()
for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids, claim_input_ids, claim_input_mask, claim_segment_ids, claim_label_ids = batch
#         ce_loss = model(input_ids, segment_ids, input_mask, label_ids)
#         cos_loss = model(claim_input_ids, claim_segment_ids, claim_input_mask, claim_label_ids)
        
#         print("start")
#         print(input_ids)
#         print(input_mask)
#         print(segment_ids)
#         print(label_ids)
#         print(claim_input_ids)
#         print(claim_input_mask)
#         print(claim_segment_ids)
#         print(claim_label_ids)
#         print("end")
    
        out_results = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask, labels=label_ids, input_ids2=claim_input_ids, token_type_ids2=claim_segment_ids, attention_mask2=claim_input_mask, labels2=claim_label_ids)
#         loss = ce_loss + cos_loss
        print("out_results:")
        print(out_results)
        loss = out_results
#         print(cos_loss)
#         print(loss.item())
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
#         if fp16 and loss_scale != 1.0:
#             # rescale loss for fp16 training
#             # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
#             loss = loss * loss_scale
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % gradient_accumulation_steps == 0:
#             if fp16 or optimize_on_cpu:
#                 if fp16 and loss_scale != 1.0:
#                     # scale down gradients for fp16 training
#                     for param in model.parameters():
#                         if param.grad is not None:
#                             param.grad.data = param.grad.data / loss_scale           
#                 is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
#                 if is_nan:
#                     logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
#                     loss_scale = loss_scale / 2
#                     model.zero_grad()
#                     continue 
#                 optimizer.step()
# #                 scheduler.step()
#                 copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
#             else:
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
#                 scheduler.step()
            model.zero_grad()
            global_step += 1

        
## v2: concat
## v3: multiply
model_to_save = model.module if hasattr(model, 'module') else model
torch.save(model.state_dict(), output_dir + "siamese_bert.pth")
torch.save(model_to_save.state_dict(), output_dir + "siamese_bert.bin")

Iteration:   0%|                                                                               | 0/438 [00:00<?, ?it/s]

logits_ce:
tensor([[-0.1326, -0.1067],
        [-0.2823,  0.3187],
        [-0.3458,  0.5263],
        [-0.5933,  0.1924],
        [-0.0010, -0.1120],
        [-0.4023, -0.1657],
        [-0.3237,  0.4646],
        [-0.4026,  0.3286],
        [-0.2223,  0.3451],
        [-0.2197,  0.0884],
        [-0.4092,  0.5253],
        [ 0.0292,  0.0604],
        [-0.3445,  0.3150],
        [-0.4460,  0.4577],
        [-0.2612, -0.3218],
        [-0.2504,  0.2688]], device='cuda:0', grad_fn=<AddmmBackward>)
loss_ce:
tensor(0.5632, device='cuda:0', grad_fn=<NllLossBackward>)
final loss:
tensor(0.5632, device='cuda:0', grad_fn=<NllLossBackward>)
out_results:
tensor(0.5632, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   0%|▏                                                                      | 1/438 [00:00<06:06,  1.19it/s]

logits_ce:
tensor([[-0.1303,  0.0827],
        [-0.4447, -0.0114],
        [-0.1851,  0.2951],
        [-0.1489,  0.2497],
        [-0.1246,  0.1251],
        [-0.1611,  0.3444],
        [-0.4634,  0.5411],
        [-0.1628, -0.0215],
        [-0.2859,  0.1155],
        [ 0.1367,  0.3415],
        [-0.5054,  0.5177],
        [-0.2765,  0.2221],
        [-0.1901, -0.2545],
        [-0.4319,  0.6899],
        [-0.3622,  0.0145],
        [-0.4846, -0.0796]], device='cuda:0', grad_fn=<AddmmBackward>)
loss_ce:
tensor(0.7352, device='cuda:0', grad_fn=<NllLossBackward>)
final loss:
tensor(0.7352, device='cuda:0', grad_fn=<NllLossBackward>)
out_results:
tensor(0.7352, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   0%|▎                                                                      | 2/438 [00:01<05:33,  1.31it/s]

...


Iteration: 100%|████████████████████████████████████████████████████████████████████▋| 436/438 [07:17<00:01,  1.09it/s]

logits_ce:
tensor([[-2.4889,  2.7254],
        [-0.3010, -0.1246],
        [ 2.8375, -3.0055],
        [-2.1361,  2.0814],
        [ 2.8662, -2.9449],
        [-2.1540,  2.1803],
        [-2.0988,  1.9751],
        [ 1.1386, -1.1486],
        [ 2.8092, -3.3866],
        [-2.3447,  2.4162],
        [-0.2905, -0.0413],
        [-2.7586,  2.7615],
        [ 2.8345, -2.8287],
        [ 1.8957, -2.0510],
        [ 0.6676, -1.0004],
        [ 3.0091, -3.2316]], device='cuda:0', grad_fn=<AddmmBackward>)
loss_ce:
tensor(0.1079, device='cuda:0', grad_fn=<NllLossBackward>)
final loss:
tensor(0.1079, device='cuda:0', grad_fn=<NllLossBackward>)
out_results:
tensor(0.1079, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration: 100%|████████████████████████████████████████████████████████████████████▊| 437/438 [07:18<00:00,  1.08it/s]

logits_ce:
tensor([[-1.4250,  1.1427],
        [ 0.5054, -0.7486],
        [ 1.8405, -1.9932],
        [ 2.4417, -2.7513],
        [ 2.1152, -2.1112],
        [ 2.7845, -3.0172],
        [ 1.8000, -2.1095],
        [-2.1621,  2.1376],
        [-2.3962,  2.2853],
        [ 2.8248, -2.9581],
        [-2.0872,  1.9816],
        [ 2.7789, -3.2022],
        [-1.8894,  1.9330],
        [ 0.2226, -0.6727],
        [-0.2110, -0.4372]], device='cuda:0', grad_fn=<AddmmBackward>)
loss_ce:
tensor(0.1074, device='cuda:0', grad_fn=<NllLossBackward>)
final loss:
tensor(0.1074, device='cuda:0', grad_fn=<NllLossBackward>)
out_results:
tensor(0.1074, device='cuda:0', grad_fn=<NllLossBackward>)



Epoch: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [19:50<00:00, 396.99s/it]


In [13]:
import csv
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
def train_and_test(data_dir, bert_model="bert-base-uncased", task_name=None,
                   output_dir=None, max_seq_length=32, do_train=False, do_eval=False, do_lower_case=False,
                   train_batch_size=32, eval_batch_size=8, learning_rate=5e-5, num_train_epochs=3,
                   warmup_proportion=0.1,no_cuda=False, local_rank=-1, seed=19, gradient_accumulation_steps=1,
                   optimize_on_cpu=False, fp16=False, loss_scale=128, saved_model=""):


    # ## Required parameters
    # parser.add_argument("--data_dir",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    # parser.add_argument("--bert_model", default=None, type=str, required=True,
    #                     help="Bert pre-trained model selected in the list: bert-base-uncased, "
    #                          "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.")
    # parser.add_argument("--task_name",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The name of the task to train.")
    # parser.add_argument("--output_dir",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The output directory where the model checkpoints will be written.")

    ## Other parameters
    # parser.add_argument("--max_seq_length",
    #                     default=128,
    #                     type=int,
    #                     help="The maximum total input sequence length after WordPiece tokenization. \n"
    #                          "Sequences longer than this will be truncated, and sequences shorter \n"
    #                          "than this will be padded.")
    # parser.add_argument("--do_train",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to run training.")
    # parser.add_argument("--do_eval",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to run eval on the dev set.")
    # parser.add_argument("--do_lower_case",
    #                     default=False,
    #                     action='store_true',
    #                     help="Set this flag if you are using an uncased model.")
    # parser.add_argument("--train_batch_size",
    #                     default=32,
    #                     type=int,
    #                     help="Total batch size for training.")
    # parser.add_argument("--eval_batch_size",
    #                     default=8,
    #                     type=int,
    #                     help="Total batch size for eval.")
    # parser.add_argument("--learning_rate",
    #                     default=5e-5,
    #                     type=float,
    #                     help="The initial learning rate for Adam.")
    # parser.add_argument("--num_train_epochs",
    #                     default=3.0,
    #                     type=float,
    #                     help="Total number of training epochs to perform.")
    # parser.add_argument("--warmup_proportion",
    #                     default=0.1,
    #                     type=float,
    #                     help="Proportion of training to perform linear learning rate warmup for. "
    #                          "E.g., 0.1 = 10%% of training.")
    # parser.add_argument("--no_cuda",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether not to use CUDA when available")
    # parser.add_argument("--local_rank",
    #                     type=int,
    #                     default=-1,
    #                     help="local_rank for distributed training on gpus")
    # parser.add_argument('--seed',
    #                     type=int,
    #                     default=42,
    #                     help="random seed for initialization")
    # parser.add_argument('--gradient_accumulation_steps',
    #                     type=int,
    #                     default=1,
    #                     help="Number of updates steps to accumulate before performing a backward/update pass.")
    # parser.add_argument('--optimize_on_cpu',
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to perform optimization and keep the optimizer averages on CPU")
    # parser.add_argument('--fp16',
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to use 16-bit float precision instead of 32-bit")
    # parser.add_argument('--loss_scale',
    #                     type=float, default=128,
    #                     help='Loss scaling, positive power of 2 values can improve fp16 convergence.')

    # args = parser.parse_args()

    processors = {
#         "cola": ColaProcessor,
#         "mnli": MnliProcessor,
        "mrpc": MrpcProcessor,
    }

    if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        device = torch.device("cuda", local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
        if fp16:
            logger.info("16-bits training currently not supported in distributed training")
            fp16 = False # (see https://github.com/pytorch/pytorch/pull/13496)
    logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, bool(local_rank != -1))

    if gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            gradient_accumulation_steps))

    train_batch_size = int(train_batch_size / gradient_accumulation_steps)

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

    if not do_train and not do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if do_train:
        if os.path.exists(output_dir) and os.listdir(output_dir):
            raise ValueError("Output directory ({}) already exists and is not emp1ty.".format(output_dir))
        os.makedirs(output_dir, exist_ok=True)

    task_name = task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()
    label_list = processor.get_labels()

#     tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=do_lower_case)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    train_examples = None
    num_train_steps = None
    if do_train:
        train_examples = processor.get_train_examples(data_dir)
        num_train_steps = int(
            len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

    # Prepare model
#     model = BertForSequenceClassification.from_pretrained(bert_model,
#                 cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(local_rank), num_labels = 2)

        model = BertForConsistencyCueClassification.from_pretrained('bert-base-uncased', num_labels=2)
        model.to(device)
        if fp16:
            model.half()

        if local_rank != -1:
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank],
                                                              output_device=local_rank)
        elif n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # Prepare optimizer
        if fp16:
            param_optimizer = [(n, param.clone().detach().to('cpu').float().requires_grad_()) \
                                for n, param in model.named_parameters()]
        elif optimize_on_cpu:
            param_optimizer = [(n, param.clone().detach().to('cpu').requires_grad_()) \
                                for n, param in model.named_parameters()]
        else:
            param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
            ]
        t_total = num_train_steps
#     print(t_total)
    if local_rank != -1:
        t_total = t_total // torch.distributed.get_world_size()
    if do_train:
        optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)

    global_step = 0
    if do_train:
        train_features = convert_examples_to_features(
            train_examples, label_list, max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", train_batch_size)
        logger.info("  Num steps = %d", num_train_steps)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        if local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

        model.train()
        for _ in trange(int(num_train_epochs), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids, = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids)
                if n_gpu > 1:
                    loss = loss.mean() # mean() to average on multi-gpu.
                if fp16 and loss_scale != 1.0:
                    # rescale loss for fp16 training
                    # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
                    loss = loss * loss_scale
                if gradient_accumulation_steps > 1:
                    loss = loss / gradient_accumulation_steps
                loss.backward()
                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % gradient_accumulation_steps == 0:
                    if fp16 or optimize_on_cpu:
                        if fp16 and loss_scale != 1.0:
                            # scale down gradients for fp16 training
                            for param in model.parameters():
                                if param.grad is not None:
                                    param.grad.data = param.grad.data / loss_scale
                        is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                        if is_nan:
                            logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                            loss_scale = loss_scale / 2
                            model.zero_grad()
                            continue
                        optimizer.step()
                        copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
                    else:
                        optimizer.step()
                    model.zero_grad()
                    global_step += 1

        torch.save(model.state_dict(), output_dir + "output.pth")


    if do_eval and (local_rank == -1 or torch.distributed.get_rank() == 0):
        eval_examples = processor.get_test_examples(data_dir)
#         eval_examples = processor.get_dev_examples(data_dir)
        claim_features = convert_claims_to_features(eval_examples, label_list, max_seq_length, tokenizer)
        eval_features = convert_pers_to_features(
            eval_examples, label_list, max_seq_length, tokenizer)
            
    
        logger.info("***** Running evaluation *****")
        logger.info("  Num examples = %d", len(eval_examples))
        logger.info("  Batch size = %d", eval_batch_size)
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
        
        claims_input_ids = torch.tensor([f.input_ids for f in claim_features], dtype=torch.long)
        claims_input_mask = torch.tensor([f.input_mask for f in claim_features], dtype=torch.long)
        claims_segment_ids = torch.tensor([f.segment_ids for f in claim_features], dtype=torch.long)
        claims_label_ids = torch.tensor([f.label_id for f in claim_features], dtype=torch.long)
        
        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, claims_input_ids, claims_input_mask, claims_segment_ids, claims_label_ids)
        # Run prediction for full data
#         eval_sampler = SequentialSampler(eval_data)
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)
#         print('all_input_ids:')
#         print(all_input_ids)
        
        

#         model.load_state_dict(torch.load(saved_model))
        model_state_dict = torch.load(saved_model)
        model = BertForConsistencyCueClassification.from_pretrained('bert-base-uncased', num_labels=2, state_dict=model_state_dict)
        model.to(device)
        
        model.eval()
        # eval_loss, eval_accuracy = 0, 0

        eval_tp, eval_pred_c, eval_gold_c = 0, 0, 0
        eval_loss, eval_macro_p, eval_macro_r = 0, 0, 0

        raw_score = []

        nb_eval_steps, nb_eval_examples = 0, 0
        for input_ids, input_mask, segment_ids, label_ids, claim_input_ids, claim_input_mask, claim_segment_ids, claim_label_ids in eval_dataloader:
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            label_ids = label_ids.to(device)
            claim_input_ids = claim_input_ids.to(device)
            claim_input_mask = claim_input_mask.to(device)
            claim_segment_ids = claim_segment_ids.to(device)
            claim_label_ids = claim_label_ids.to(device)

#             print("start")
#             print(input_ids)
#             print(input_mask)
#             print(segment_ids)
#             print(label_ids)
#             print(claim_input_ids)
#             print(claim_input_mask)
#             print(claim_segment_ids)
#             print(claim_label_ids)
#             print("end")
            with torch.no_grad():
                tmp_eval_loss = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask, labels=label_ids, input_ids2=claim_input_ids, token_type_ids2=claim_segment_ids, attention_mask2=claim_input_mask, labels2=claim_label_ids)
                
                logits = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask, input_ids2=claim_input_ids, token_type_ids2=claim_segment_ids, attention_mask2=claim_input_mask)
            
            print(logits)
#             print(logits[0])
            logits = logits.detach().cpu().numpy()
            print(logits)
            label_ids = label_ids.to('cpu').numpy()
#             print(label_ids)

            # Micro F1 (aggregated tp, fp, fn counts across all examples)
            tmp_tp, tmp_pred_c, tmp_gold_c = tp_pcount_gcount(logits, label_ids)
            eval_tp += tmp_tp
            eval_pred_c += tmp_pred_c
            eval_gold_c += tmp_gold_c
            
            pred_label = np.argmax(logits, axis=1)
            raw_score += zip(logits, pred_label, label_ids)
            
            # Macro F1 (averaged P, R across mini batches)
            tmp_eval_p, tmp_eval_r, tmp_eval_f1 = p_r_f1(logits, label_ids)

            eval_macro_p += tmp_eval_p
            eval_macro_r += tmp_eval_r

            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_examples += input_ids.size(0)
            nb_eval_steps += 1


        # Micro F1 (aggregated tp, fp, fn counts across all examples)
        eval_micro_p = eval_tp / eval_pred_c
        eval_micro_r = eval_tp / eval_gold_c
        eval_micro_f1 = 2 * eval_micro_p * eval_micro_r / (eval_micro_p + eval_micro_r)

        # Macro F1 (averaged P, R across mini batches)
        eval_macro_p = eval_macro_p / nb_eval_steps
        eval_macro_r = eval_macro_r / nb_eval_steps
        eval_macro_f1 = 2 * eval_macro_p * eval_macro_r / (eval_macro_p + eval_macro_r)

        eval_loss = eval_loss / nb_eval_steps
        result = {
                  'eval_loss': eval_loss,
                  'eval_micro_p': eval_micro_p,
                  'eval_micro_r': eval_micro_r,
                  'eval_micro_f1': eval_micro_f1,
                  'eval_macro_p': eval_macro_p,
                  'eval_macro_r': eval_macro_r,
                  'eval_macro_f1': eval_macro_f1,
#                   'global_step': global_step,
#                   'loss': tr_loss/nb_tr_steps
                  }

        output_eval_file = os.path.join(output_dir, "siamesebert_test_eval_results.txt")
        output_raw_score = os.path.join(output_dir, "siamesebert_test_raw_score.csv")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

        with open(output_raw_score, 'w') as fout:
            fields = ["undermine_score", "support_score","predict_label", "gold"]
            writer = csv.DictWriter(fout, fieldnames=fields)
            writer.writeheader()
            for score, pred, gold in raw_score:
                writer.writerow({
                    "undermine_score": str(score[0]),
                    "support_score": str(score[1]),
                    "predict_label": str(pred),
                    "gold": str(gold)
                })

In [ ]:
def experiments():
    data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
#     data_dir = "/home/syg340/Dataset/"

    # data_dir_output = data_dir + "output2/"
    data_dir_output = "D:/Projects/Stance/Models/"
    train_and_test(data_dir=data_dir, do_train=True, do_eval=True, output_dir=data_dir_output,task_name="Mrpc")


In [14]:
def evaluation_with_pretrained():
    bert_model = "D:/Projects/Stance/Models/Consistency_Cues/siamese_bert.pth"
    data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
    # data_dir_output = data_dir + "output2/"
    ## v2: concat
    ## v3: multiply
    data_dir_output = "D:/Projects/Stance/Evaluation/bert_dummy_output/siamese_bert"
    train_and_test(data_dir=data_dir, do_train=False, do_eval=True, output_dir=data_dir_output,task_name="mrpc",saved_model=bert_model)

In [15]:
if __name__ == "__main__":
#     experiments()
    evaluation_with_pretrained()

05/27/2020 23:27:57 - INFO - run_classifier -   device cuda n_gpu 1 distributed training False
05/27/2020 23:28:07 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\arsen\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/27/2020 23:28:07 - INFO - run_classifier -   *** Claim Example ***
05/27/2020 23:28:07 - INFO - run_classifier -   guid: test-1
05/27/2020 23:28:07 - INFO - run_classifier -   tokens: [CLS] school day should be extended [SEP]
05/27/2020 23:28:07 - INFO - run_classifier -   input_ids: 101 2082 2154 2323 2022 3668 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/27/2020 23:28:07 - INFO - run_classifier -   input_mask: 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/27/2020 23:28:07 - INFO - run_classifier -   segment_ids: 0 0 0 

05/27/2020 23:28:36 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



logits_ce:
tensor([[ 0.0648, -0.4265],
        [ 1.5314, -1.5976],
        [-1.8795,  1.7026],
        [-1.9548,  1.7958],
        [-1.2224,  0.8261],
        [-2.2591,  2.2422],
        [-2.2512,  2.2261],
        [-2.1247,  2.0389]], device='cuda:0')
loss_ce:
tensor(0.1526, device='cuda:0')
final loss:
tensor(0.1526, device='cuda:0')
logits_ce:
tensor([[ 0.0648, -0.4265],
        [ 1.5314, -1.5976],
        [-1.8795,  1.7026],
        [-1.9548,  1.7958],
        [-1.2224,  0.8261],
        [-2.2591,  2.2422],
        [-2.2512,  2.2261],
        [-2.1247,  2.0389]], device='cuda:0')
tensor([[ 0.0648, -0.4265],
        [ 1.5314, -1.5976],
        [-1.8795,  1.7026],
        [-1.9548,  1.7958],
        [-1.2224,  0.8261],
        [-2.2591,  2.2422],
        [-2.2512,  2.2261],
        [-2.1247,  2.0389]], device='cuda:0')
[[ 0.06475578 -0.42653236]
 [ 1.5314255  -1.5976137 ]
 [-1.8795117   1.702599  ]
 [-1.9547505   1.7957938 ]
 [-1.2224255   0.82612073]
 [-2.2591374   2.2421634 ]
 [-2.

logits_ce:
tensor([[ 2.1439, -2.3066],
        [-2.7099,  3.1196],
        [-2.3744,  2.3963],
        [-2.7687,  3.1998],
        [-2.7723,  3.2697],
        [-2.7484,  3.3437],
        [-2.5618,  2.8389],
        [ 2.5897, -2.7922]], device='cuda:0')
loss_ce:
tensor(0.7336, device='cuda:0')
final loss:
tensor(0.7336, device='cuda:0')
logits_ce:
tensor([[ 2.1439, -2.3066],
        [-2.7099,  3.1196],
        [-2.3744,  2.3963],
        [-2.7687,  3.1998],
        [-2.7723,  3.2697],
        [-2.7484,  3.3437],
        [-2.5618,  2.8389],
        [ 2.5897, -2.7922]], device='cuda:0')
tensor([[ 2.1439, -2.3066],
        [-2.7099,  3.1196],
        [-2.3744,  2.3963],
        [-2.7687,  3.1998],
        [-2.7723,  3.2697],
        [-2.7484,  3.3437],
        [-2.5618,  2.8389],
        [ 2.5897, -2.7922]], device='cuda:0')
[[ 2.14385   -2.306623 ]
 [-2.709866   3.1196296]
 [-2.3744247  2.3963056]
 [-2.7686725  3.1998403]
 [-2.772263   3.2696958]
 [-2.7484322  3.3437035]
 [-2.561831   2.8

tensor([[-1.8200,  1.5858],
        [-1.0329,  0.6193],
        [-1.8111,  1.5539],
        [ 1.3459, -1.3739],
        [ 0.8904, -1.0289],
        [ 1.0153, -1.1600],
        [-0.6290,  0.1689],
        [-0.8732,  0.4027]], device='cuda:0')
[[-1.8199552   1.5857679 ]
 [-1.0329292   0.6193138 ]
 [-1.8111484   1.5539362 ]
 [ 1.345947   -1.3739398 ]
 [ 0.8904395  -1.0288501 ]
 [ 1.0153131  -1.1600018 ]
 [-0.62899053  0.16890657]
 [-0.87320614  0.4027464 ]]
logits_ce:
tensor([[-1.3789,  1.0365],
        [-2.3887,  2.4452],
        [-1.9086,  1.7366],
        [-2.3546,  2.4048],
        [-0.6712,  0.2078],
        [ 0.1882, -0.6023],
        [-1.6257,  1.3552],
        [-2.1696,  2.0449]], device='cuda:0')
loss_ce:
tensor(0.5149, device='cuda:0')
final loss:
tensor(0.5149, device='cuda:0')
logits_ce:
tensor([[-1.3789,  1.0365],
        [-2.3887,  2.4452],
        [-1.9086,  1.7366],
        [-2.3546,  2.4048],
        [-0.6712,  0.2078],
        [ 0.1882, -0.6023],
        [-1.6257,  1.355

...

logits_ce:
tensor([[ 0.5342, -0.7389],
        [-1.5051,  1.2305],
        [-1.6179,  1.3417],
        [ 1.6355, -1.6831],
        [-2.0308,  1.8951],
        [-1.6562,  1.3861],
        [ 0.7684, -0.8912],
        [ 2.6419, -2.8709]], device='cuda:0')
tensor([[ 0.5342, -0.7389],
        [-1.5051,  1.2305],
        [-1.6179,  1.3417],
        [ 1.6355, -1.6831],
        [-2.0308,  1.8951],
        [-1.6562,  1.3861],
        [ 0.7684, -0.8912],
        [ 2.6419, -2.8709]], device='cuda:0')
[[ 0.5341889  -0.73887783]
 [-1.5051264   1.2305154 ]
 [-1.6179405   1.3417001 ]
 [ 1.6354903  -1.6831388 ]
 [-2.0307517   1.8950679 ]
 [-1.6562364   1.3860519 ]
 [ 0.76843214 -0.89119464]
 [ 2.6419454  -2.8708901 ]]
logits_ce:
tensor([[ 1.0919, -1.1037],
        [-1.0383,  0.6718],
        [ 0.8302, -0.9500],
        [-2.0977,  1.9962],
        [-1.5224,  1.1932],
        [ 0.6808, -0.8313],
        [-2.2246,  2.1946],
        [-2.6397,  2.8746]], device='cuda:0')
loss_ce:
tensor(1.5211, device='cud

logits_ce:
tensor([[-0.1742, -0.2201],
        [ 0.1451, -0.4959],
        [ 1.2571, -1.2784],
        [-1.1806,  0.8385],
        [-0.4441,  0.0212],
        [-1.7425,  1.4917],
        [ 2.7521, -2.9721],
        [-2.0514,  1.8738]], device='cuda:0')
tensor([[-0.1742, -0.2201],
        [ 0.1451, -0.4959],
        [ 1.2571, -1.2784],
        [-1.1806,  0.8385],
        [-0.4441,  0.0212],
        [-1.7425,  1.4917],
        [ 2.7521, -2.9721],
        [-2.0514,  1.8738]], device='cuda:0')
[[-0.17424166 -0.22014695]
 [ 0.14506516 -0.49590358]
 [ 1.2571172  -1.2784364 ]
 [-1.1806059   0.83854795]
 [-0.44414356  0.02115753]
 [-1.742538    1.4916896 ]
 [ 2.7521484  -2.9720752 ]
 [-2.0514493   1.8737969 ]]
logits_ce:
tensor([[-1.4060,  1.0351],
        [ 2.9183, -3.3348],
        [ 3.0411, -3.3869],
        [ 3.0235, -3.2937],
        [-1.6565,  1.3584],
        [-2.5700,  2.6883],
        [-2.1555,  2.0473],
        [-2.1777,  2.0658]], device='cuda:0')
loss_ce:
tensor(0.3265, device='cud

logits_ce:
tensor([[-1.8475,  1.6358],
        [-0.3788, -0.0397],
        [-0.2360, -0.1987],
        [-2.3525,  2.4180],
        [ 0.0582, -0.5335],
        [-1.1602,  0.7778],
        [-1.4251,  1.1585],
        [-1.8835,  1.7761]], device='cuda:0')
tensor([[-1.8475,  1.6358],
        [-0.3788, -0.0397],
        [-0.2360, -0.1987],
        [-2.3525,  2.4180],
        [ 0.0582, -0.5335],
        [-1.1602,  0.7778],
        [-1.4251,  1.1585],
        [-1.8835,  1.7761]], device='cuda:0')
[[-1.8475416   1.635762  ]
 [-0.3787804  -0.0397494 ]
 [-0.23603274 -0.19869421]
 [-2.3524609   2.4179575 ]
 [ 0.05818545 -0.53345585]
 [-1.1602283   0.7778154 ]
 [-1.4250596   1.1585265 ]
 [-1.8835435   1.7761323 ]]
logits_ce:
tensor([[ 0.5810, -0.8731],
        [-2.0820,  1.9554],
        [-0.6462,  0.2036],
        [ 2.1818, -2.3015],
        [ 2.5954, -2.8428],
        [ 0.8052, -1.0212],
        [-1.3257,  0.9718],
        [-1.1766,  0.7980]], device='cuda:0')
loss_ce:
tensor(2.3060, device='cud

05/27/2020 23:29:51 - INFO - run_classifier -   ***** Eval results *****
05/27/2020 23:29:51 - INFO - run_classifier -     eval_loss = 1.0964370662888636
05/27/2020 23:29:51 - INFO - run_classifier -     eval_macro_f1 = 0.667715143314651
05/27/2020 23:29:51 - INFO - run_classifier -     eval_macro_p = 0.6494888156991903
05/27/2020 23:29:51 - INFO - run_classifier -     eval_macro_r = 0.6869939618498702
05/27/2020 23:29:51 - INFO - run_classifier -     eval_micro_f1 = 0.6635167060411745
05/27/2020 23:29:51 - INFO - run_classifier -     eval_micro_p = 0.6588471849865952
05/27/2020 23:29:51 - INFO - run_classifier -     eval_micro_r = 0.6682528891910265


logits_ce:
tensor([[-0.1994, -0.1918],
        [-1.6059,  1.2911],
        [ 1.7316, -1.8410],
        [ 1.8512, -1.9644],
        [ 1.4590, -1.5283]], device='cuda:0')
tensor([[-0.1994, -0.1918],
        [-1.6059,  1.2911],
        [ 1.7316, -1.8410],
        [ 1.8512, -1.9644],
        [ 1.4590, -1.5283]], device='cuda:0')
[[-0.19937393 -0.19179516]
 [-1.6059208   1.291123  ]
 [ 1.7315972  -1.841001  ]
 [ 1.8512089  -1.9643966 ]
 [ 1.4590257  -1.5282884 ]]
